In [ ]:
# from zipfile import ZipFile

# file_name = "data.zip"

# with ZipFile(file_name, 'r') as zip:
#   zip.extractall()
#   print('Done')

# **LIBRARIES**

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.cluster import SpectralClustering
from numpy import random
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.preprocessing import normalize
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import kneighbors_graph
import networkx as nx
from scipy.optimize import linear_sum_assignment

# **LOAD TRAINING AND TESTING DATASETS FOR KMEANS**


In [ ]:
# datasets = tfds.load('kddcup99', split=['train', 'test'])

# # Calculate the cardinalities
# train_cardinality = tf.data.experimental.cardinality(datasets[0]).numpy()
# test_cardinality = tf.data.experimental.cardinality(datasets[1]).numpy()

# print(f'Train Cardinality: {train_cardinality}') # 4,898,431
# print(f'Test Cardinality: {test_cardinality}') # 311,029

# percent_10 = 494021
# training_df = tfds.as_dataframe(datasets[0].shuffle(buffer_size=1024).take(percent_10))
# testing_df = tfds.as_dataframe(datasets[1])

Train Cardinality: 4898431
Test Cardinality: 311029


In [ ]:
# training_df = training_df.drop_duplicates()
# testing_df = testing_df.drop_duplicates()

# training_labels = np.unique(training_df['label'].values)
# testing_df = testing_df[testing_df.label.isin(training_labels)]

# training_data = np.array(training_df.drop(['label'], axis=1).values.tolist())
# testing_data = np.array(testing_df.drop(['label'], axis=1).values.tolist())

# scaler = MinMaxScaler(feature_range=(0, 1))

# training_data = scaler.fit_transform(training_data)
# testing_data = scaler.fit_transform(testing_data)

# true_labels = testing_df['label'].values.tolist()
# ks = [7, 15, 23, 31, 45]

# **LOAD TRAINING AND TESTING DATASETS FOR SPECTRAL**


In [ ]:
# datasets = tfds.load('kddcup99', split=['train', 'test'])
# samples_per_class = int(4898431 * 0.005 / len(classes))

# from functools import reduce
# datasets_per_class = []

# for i in classes:
#     class_dataset = datasets[0].filter(lambda x: x["label"] == i)
#     class_dataset = class_dataset.map(lambda x: {k: v for k, v in x.items() if k != 'label'})
#     class_dataset = class_dataset.take(samples_per_class)
#     datasets_per_class.append(class_dataset)

# import csv

# for i in range(10, 23):
#   data = []
#   index = i
#   file_name = 'class'+str(index)+'.csv'
#   for i, training_tf_class in enumerate(datasets_per_class):
#     if i == index:
#       data.extend([list(sample.values()) for sample in tfds.as_numpy(training_tf_class.take(samples_per_class))])

#   with open(file_name, mode='w') as file:
#       writer = csv.writer(file)
#       for row in data:
#           writer.writerow(row)

# **FUNCTIONS**

In [ ]:
def kmeans_plus_plus(data, k):
    centroids = [data[np.random.randint(data.shape[0])]]
    for _ in range(k - 1):
        distances = np.min(np.sum((data[:, np.newaxis] - centroids) ** 2, axis=2), axis=1)
        probabilities = distances / distances.sum()
        cumulative_probabilities = probabilities.cumsum()
        r = np.random.rand()
        i = random.randint(0, len(data))
        for j, p in enumerate(cumulative_probabilities):
            if r < p:
                i = j
                break
        centroids.append(data[i])
    return centroids

In [ ]:
def k_means_clustering(data, init_centroids, k, max_iterations):

    centroids = init_centroids
    old_centroids = None

    iterations = 0
    empty_flag = False
    while (not compare_centroids(old_centroids, centroids)) and (
        iterations < max_iterations
    ):
        old_centroids = centroids
        labels = get_labels(data, centroids)
        centroids, flag = get_centroids(data, labels, k)
        empty_flag = flag
        while empty_flag:
          labels = get_labels(data, centroids)
          centroids, flag = get_centroids(data, labels, k)
          empty_flag = flag
        iterations += 1
    return centroids


def compare_centroids(old_centroids, centroids):
    if old_centroids is not None:
        return np.array_equal(old_centroids, centroids)
    else:
        return False


def get_labels(data, centroids):
    distances = [
        [np.linalg.norm(data[j] - centroids[i]) for i in range(len(centroids))]
        for j in range(len(data))
    ]
    return np.argmin(distances, axis=1)


def get_centroids(data, labels, k):
    empty_flag = False
    new_centroids = []
    for i in range(k):
        cluster = []
        for j in range(len(data)):
            if labels[j] == i:
                cluster.append(data[j])
        if len(cluster) > 0:
            new_centroid = np.mean(cluster, axis=0).reshape((1, -1)).squeeze()
            new_centroids.append(new_centroid.tolist())
        # when a centroid is empty
        else:
            empty_flag = True
            while True:
                # kmeans++ re-initialize
                new_centroid = kmeans_plus_plus(data, 1)
                # check existance
                if not new_centroid in new_centroids:
                    break
            new_centroids.append(new_centroid.tolist())
    return np.array(new_centroids), empty_flag

In [ ]:
def get_row_sum(matrix, row_index):
    row = matrix[row_index]
    return sum(row)


def get_col_sum(matrix, column_index):
    column = [row[column_index] for row in matrix]
    return sum(column)


def get_contingency_matrix(clusters_labels, classes):
    return np.array([[clusters_labels[i].count(x) for x in classes] for i in range(k)])


def get_recall_precision(contingency_matrix):
    cluster_max = [max(row) for row in contingency_matrix]

    precision = []
    recall = []
    for i in range(contingency_matrix.shape[0]):
        index = np.where(contingency_matrix == cluster_max[i])
        index = [index[0][0], index[1][0]]
        col_sum = get_col_sum(contingency_matrix, index[1])
        row_sum = get_row_sum(contingency_matrix, index[0])
        precision.append(cluster_max[i] / row_sum)
        recall.append(cluster_max[i] / col_sum)
    return recall, precision


def get_f1_score(recalls, precisions):
    f1s = []
    for recall, precision in zip(recalls, precisions):
        if recall == 0 or precision == 0:
            f1 = 0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)
        f1s.append(f1)
    sum = 0
    count = 0
    for score in f1s:
        if score != 0:
            sum += score
            count += 1
    return f1s, sum / count


def get_conditional_entropy(contingency_matrix):

    cluster_specifics = []
    conditional_entropy = 0
    sum = contingency_matrix.sum()

    for i in range(contingency_matrix.shape[0]):
        row_sum = get_row_sum(contingency_matrix, i)
        cluster_specific = 0
        for j in range(contingency_matrix.shape[1]):
            p = contingency_matrix[i][j] / row_sum
            if p != 0:
                cluster_specific -= p * math.log2(p)
        cluster_specifics.append(cluster_specific)

    for i, entropy in enumerate(cluster_specifics):
        row_sum = get_row_sum(contingency_matrix, i)
        p = row_sum / sum
        conditional_entropy += p * entropy
    return cluster_specifics, conditional_entropy

def get_confusion_matrix(contingency_matrix):
  # calculate true positive TP
  TP = 0
  for i in range(contingency_matrix.shape[0]):
    for j in range(contingency_matrix.shape[1]):
      element = contingency_matrix[i][j]
      if element >= 2:
        TP += math.comb(element, 2)

  # calculate false positive FP
  FP = 0
  for i in range(contingency_matrix.shape[0]):
    row_sum = get_row_sum(contingency_matrix, i)
    FP += math.comb(row_sum, 2)
  FP -= TP

  # calculate false negative FN
  FN = 0
  cluster_max = [max(row) for row in contingency_matrix]
  for i in range(contingency_matrix.shape[0]):
    index = np.where(contingency_matrix == cluster_max[i])
    index = [index[0][0], index[1][0]]
    col_sum = get_col_sum(contingency_matrix, index[1])
    FN += math.comb(col_sum, 2)
  FN -= TP

  # calculate true negative TN
  TN = 0
  N = math.comb(contingency_matrix.sum(), 2)
  TN = N - TP - FP - FN

  return np.array([[TP, FP], [FN, TN]])


def get_rand_index(confusion_matrix):
  return np.trace(confusion_matrix)/confusion_matrix.sum()


def get_jaccard_index(confusion_matrix):
  return confusion_matrix[0][0]/(confusion_matrix.sum() - confusion_matrix[1][1])


def get_precision(confusion_matrix):
  return confusion_matrix[0][0]/(confusion_matrix[0][0] + confusion_matrix[0][1])


def get_recall(confusion_matrix):
  return confusion_matrix[0][0]/(confusion_matrix[0][0] + confusion_matrix[1][0])


def get_maximum_matching(contingency_matrix):
    row_ind, col_ind = linear_sum_assignment(contingency_matrix, maximize=True)
    reordered_matrix = contingency_matrix[row_ind, :][:, col_ind]
    return np.trace(reordered_matrix)/contingency_matrix.sum(), reordered_matrix

# **DATA PRE-PROCESSING FOR KMEANS CLUSTERING**

In [ ]:
# training_df = training_df.drop_duplicates()
# testing_df = testing_df.drop_duplicates()

# training_labels = np.unique(training_df['label'].values)
# testing_df = testing_df[testing_df.label.isin(training_labels)]

# training_data = np.array(training_df.drop(['label'], axis=1).values.tolist())
# testing_data = np.array(testing_df.drop(['label'], axis=1).values.tolist())

# scaler = MinMaxScaler(feature_range=(0, 1))

# training_data = scaler.fit_transform(training_data)
# testing_data = scaler.fit_transform(testing_data)

# true_labels = testing_df['label'].values.tolist()
# ks = [7, 15, 23, 31, 45]

# **DATA PRE-PROCESSING FOR SPECTRAL CLUSTERING**

In [ ]:
classes = [27, 16, 14, 7, 25, 20, 34, 15, 9, 1, 19, 32, 8, 18, 4, 3, 12, 30, 35, 6, 2, 23, 17]
samples_per_class = [1064,1064,1064,1064,1064,1064,1020,1064,9,1064,264,979,21,4, 53 ,8 ,7 ,2,20,12,30,10,3]

without = [44,1051,1033,604,640,426,893,716,9,511,206,918,19,4,53,8,7,2,20,12,30,10,3]

training_data = []
true_labels = []
for i in range(len(classes)):
  data = []
  df = pd.read_csv('data/class'+str(i)+'.csv', header=None)
  for sample in df.to_numpy():
    data.append(sample)
  data = [list(x) for x in set(tuple(x) for x in data)]
  true_labels.extend([classes[i]] * len(data))
  for sample in data:
    training_data.append(sample)

import random

zipped = list(zip(training_data, true_labels))

random.seed(42)
random.shuffle(zipped)

training_data, true_labels = zip(*zipped)

scaler = MinMaxScaler(feature_range=(0, 1))
training_data = scaler.fit_transform(training_data)

# **SPECTRAL CLUSTERING**

**RBF SIMILARITY**

In [ ]:
# gamma = 0.5
# similarity_matrix = rbf_kernel(training_data, gamma = gamma)

**NEAREST NEIGHBORS SIMILARITY**

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components

def check_adjacency_matrix(data, init_neighbors):
  neighbor = init_neighbors
  while True:
    similarity_matrix = kneighbors_graph(data, n_neighbors=neighbor).toarray()
    csr = csr_matrix(similarity_matrix)
    n_components, component_labels = connected_components(csr)
    print(f'n_components: {n_components} neighbor: {neighbor}')
    if n_components == 1:
      break
    neighbor += 1

# check_adjacency_matrix(training_data, 40)

similarity_matrix = kneighbors_graph(training_data, n_neighbors=43).toarray()

In [ ]:
degree_matrix = np.diag(np.sum(similarity_matrix, axis=1))
laplacian_matrix = degree_matrix - similarity_matrix
asymmetric_laplacian = np.linalg.inv(degree_matrix) @ laplacian_matrix

eig_vals, eig_vecs = np.linalg.eigh(asymmetric_laplacian)

# sort eigen values in ascending order
idxs = eig_vals.argsort()
eig_vals = eig_vals[idxs]
eig_vecs = eig_vecs[:, idxs]

k_vectors = 0
for i, val in enumerate(eig_vals):
  if val > 0:
    k_vectors = i
    break

U = eig_vecs[:, :k_vectors+1]
Y = normalize(U)

# **CLUSTERING**

In [ ]:
# kmeans
# used_data = training_data

# spectral
used_data = Y
testing_data = Y

k = 23
init_centroids = kmeans_plus_plus(used_data, k)
centroids = k_means_clustering(used_data, init_centroids, k, 10)

# find cluster of each sample
# indexes identifies sample
# value at each index identifies cluster of that sample
x = get_labels(testing_data, centroids)

# **USING SKLEARN**

In [ ]:
# from sklearn.cluster import KMeans

# k = 23
# kmeans = KMeans(n_clusters=k, max_iter=15, n_init=1)
# kmeans.fit(training_data)

# x = kmeans.predict(training_data)

In [ ]:
# from sklearn.cluster import SpectralClustering

# k = 23
# # clustering = SpectralClustering(n_clusters=k, random_state=0, affinity='nearest_neighbors', n_neighbors=43)
# clustering = SpectralClustering(n_clusters=k, random_state=0, gamma=0.5)

# # x = clustering.fit(training_data)
# x = clustering.fit_predict(training_data)

In [ ]:
# x.affinity_matrix_

# **CONTINGENCY MATRIX**

In [ ]:
# get samples at each cluster
# testing_clusters variable is formed of a list of lists
# each list represent a cluster
# contents of each cluster represent index of sample in testing data
testing_clusters = []
for i in range(k):
    testing_clusters.append([])

for i in range(k):
    for j in range(len(testing_data)):
        if x[j] == i:
            testing_clusters[i].append(j)

# replace samples indexes in each cluster by their true label
# pred_labels variable is a list of lists
# each list represent a cluster and its contents is the true labels of samples
# in that cluster
clusters_labels = [[true_labels[i] for i in cluster] for cluster in testing_clusters]

classes = np.unique(true_labels).tolist()
contingency_matrix = get_contingency_matrix(clusters_labels, classes)

# **SCORES**

In [ ]:
recall, precision = get_recall_precision(contingency_matrix)
f1_scores, f1_score = get_f1_score(recall, precision)
max_matching, reordered_matrix = get_maximum_matching(contingency_matrix)
specifics, conditional_entropy = get_conditional_entropy(contingency_matrix)
confusion_matrix = get_confusion_matrix(contingency_matrix)
rand_index = get_rand_index(confusion_matrix)
jaccard_index = get_jaccard_index(confusion_matrix)
confusion_precision = get_precision(confusion_matrix)
confusion_recall = get_recall(confusion_matrix)
# print(f"contingency matrix:\n {contingency_matrix}")ass
print(f'Precision Mean: {np.mean(precision, axis=0)}')
print(f'Recall Mean: {np.mean(recall, axis=0)}')
print(f"F1 score: {f1_score}")
# print(f"Specifics: {specifics}")
# print(f"Reordered Matrix:\n {reordered_matrix}")
print(f"Maximum Matching: {max_matching}")
print(f"Conditional Entropy: {conditional_entropy}")
print(f"Confusion Matrix:\n {confusion_matrix}")
print(f"Rand Index: {rand_index}")
print(f"Jaccard Index: {jaccard_index}")
print(f"Confusion Precison: {confusion_precision}")
print(f"Confusion Recall: {confusion_recall}")

In [ ]:
np.sum(contingency_matrix, axis=1)